## Pull and clean all edge data (for 1 year) as test

Note: actual data pulling was done with pull_raw_data.py so it could be done more easily on a UNIX cluster

In [1]:
import pandas as pd
from sportsref import euro

year = 2017

In [2]:
schedules = []

for league in euro.LEAGUE_IDS:
    season = euro.Season(year, league)
    
    # Note: Regular season only used, as playoff games are also regular season Euroleague/Eurocup games
    schedules.append(season.schedule(kind='R'))
    
schedule_df = pd.concat(schedules)
schedule_df.reset_index(drop=True)

,boxscore_id,visitor_team_name,visitor_pts,home_team_name,home_pts,overtimes,notes,is_playoffs
0,2016-10-08-aris,Aries Trikala,65.0,Aris,75.0,NaN,NaN,False
1,2016-10-08-doxa-bc,P.A.O.K.,78.0,Doxa Lefkadas,84.0,NaN,NaN,False
2,2016-10-08-koroivos,Olympiacos,88.0,Koroivos,66.0,NaN,NaN,False
3,2016-10-08-lavrio,Kymis,69.0,Lavrio B.C.,75.0,NaN,NaN,False
4,2016-10-08-rhodes,Rethymno Aegean,74.0,Kolossos Rodou,66.0,NaN,NaN,False
5,2016-10-09-apollon-patras,AEK Athens,77.0,Apollon Patras,59.0,NaN,NaN,False
6,2016-10-09-panathinaikos,Promitheas Patras,71.0,Panathinaikos Superfoods,81.0,NaN,NaN,False
7,2016-10-15-aek-athens,Lavrio B.C.,56.0,AEK Athens,96.0,NaN,NaN,False
8,2016-10-15-aries-trikala,Apollon Patras,60.0,Aries Trikala,65.0,NaN,NaN,False
9,2016-10-15-paok,Kolossos Rodou,73.0,P.A.O.K.,85.0,NaN,NaN,False


### Remove duplicate games if necessary

In [3]:
print(len(schedule_df))

schedule_df.drop_duplicates(subset=['boxscore_id'], inplace=True)

print(len(schedule_df))

1475
1475


## Create new df with players vs teams (by pulling boxscores)

Note: only using boxscore_id from schedule_df because schedule_df data is unreliable


In [4]:
player_dfs_list = []

for index, row in schedule_df.iterrows():
    print(str(index) + ' ' + row['boxscore_id'])
    
    # commented out for safety
    #box = euro.BoxScore(row['boxscore_id'])
    
    # home specific
    home_df = box.get_home_stats()
    home_df['team_id'] = box.home()
    home_df['opp_id'] = box.away()
    home_df['team_pts'] = box.home_score()
    home_df['opp_pts'] = box.away_score()
   
    # away specific
    away_df = box.get_away_stats()
    away_df['team_id'] = box.away()
    away_df['opp_id'] = box.home()
    away_df['team_pts'] = box.away_score()
    away_df['opp_pts'] = box.home_score()
    
    both_df = pd.concat([home_df, away_df], ignore_index=True)
    
    # general
    both_df['boxscore_id'] = row['boxscore_id']
    both_df['is_playoffs'] = row['is_playoffs']
    
    player_dfs_list.append(both_df)

0 2016-10-08-aris
1 2016-10-08-doxa-bc
2 2016-10-08-koroivos
3 2016-10-08-lavrio
4 2016-10-08-rhodes
5 2016-10-09-apollon-patras
6 2016-10-09-panathinaikos
7 2016-10-15-aek-athens
8 2016-10-15-aries-trikala
9 2016-10-15-paok
10 2016-10-15-promitheas
11 2016-10-15-rethymno
12 2016-10-17-kymis-bc
13 2016-10-17-olympiakos
14 2016-10-22-apollon-patras
15 2016-10-22-aris
16 2016-10-22-doxa-bc
17 2016-10-22-kymis-bc
18 2016-10-22-lavrio
19 2016-10-22-rhodes
20 2016-10-23-panathinaikos
21 2016-10-29-aries-trikala
22 2016-10-29-koroivos
23 2016-10-29-paok
24 2016-10-29-promitheas
25 2016-10-29-rethymno
26 2016-10-30-olympiakos
27 2016-10-31-panathinaikos
28 2016-11-05-aek-athens
29 2016-11-05-apollon-patras
30 2016-11-05-lavrio
31 2016-11-05-rhodes
32 2016-11-06-kymis-bc
33 2016-11-07-aris
34 2016-11-07-doxa-bc
35 2016-11-12-doxa-bc
36 2016-11-12-koroivos
37 2016-11-12-paok
38 2016-11-12-promitheas
39 2016-11-12-rethymno
40 2016-11-13-olympiakos
41 2016-11-13-panathinaikos
42 2016-11-19-aek-at

129 2017-02-28-hapoel-jerusalem
130 2017-02-28-kuban
131 2017-02-28-valencia
132 2017-03-03-gran-canaria
133 2017-03-03-khimki
134 2017-03-03-triumph-moscow
135 2017-03-03-unicaja-malaga
136 2017-03-08-bayern-muenchen
137 2017-03-08-valencia
138 2017-03-14-kuban
139 2017-03-14-valencia
140 2017-03-17-hapoel-jerusalem
141 2017-03-17-unicaja-malaga
142 2017-03-22-valencia
143 2017-03-28-valencia
144 2017-03-31-unicaja-malaga
145 2017-04-05-valencia
0 2016-09-30-real-madrid
1 2016-10-01-estudiantes
2 2016-10-01-obradoiro
3 2016-10-02-fuenlabrada
4 2016-10-02-gran-canaria
5 2016-10-02-manresa
6 2016-10-02-murcia
7 2016-10-02-zaragoza
8 2016-10-06-andorra
9 2016-10-06-sevilla
10 2016-10-06-valencia
11 2016-10-07-bilbao
12 2016-10-07-canarias
13 2016-10-07-joventut
14 2016-10-07-unicaja-malaga
15 2016-10-07-vitoria
16 2016-10-09-andorra
17 2016-10-09-barcelona
18 2016-10-09-canarias
19 2016-10-09-fuenlabrada
20 2016-10-09-gran-canaria
21 2016-10-09-murcia
22 2016-10-09-obradoiro
23 2016-10-0

26 2016-10-23-turin
27 2016-10-23-varese
28 2016-10-24-caserta
29 2016-10-30-brindisi
30 2016-10-30-milano
31 2016-10-30-orlandina
32 2016-10-30-pistoia
33 2016-10-30-sassari
34 2016-10-30-trento
35 2016-10-30-venezia
36 2016-11-05-brescia
37 2016-11-05-pistoia
38 2016-11-06-caserta
39 2016-11-06-pesaro
40 2016-11-06-reggio-emilia
41 2016-11-06-turin
42 2016-11-06-varese
43 2016-11-07-cremona
44 2016-11-13-avellino
45 2016-11-13-cantu
46 2016-11-13-milano
47 2016-11-13-orlandina
48 2016-11-13-pesaro
49 2016-11-13-venezia
50 2016-11-14-trento
51 2016-11-19-varese
52 2016-11-20-caserta
53 2016-11-20-cremona
54 2016-11-20-pistoia
55 2016-11-20-reggio-emilia
56 2016-11-20-sassari
57 2016-11-20-turin
58 2016-11-21-brindisi
59 2016-11-26-sassari
60 2016-11-27-avellino
61 2016-11-27-cantu
62 2016-11-27-milano
63 2016-11-27-orlandina
64 2016-11-27-pesaro
65 2016-11-27-venezia
66 2016-12-03-varese
67 2016-12-04-avellino
68 2016-12-04-brindisi
69 2016-12-04-cremona
70 2016-12-04-pistoia
71 2016-

132 2017-01-13-paris-levallois
133 2017-01-14-chalons-en-champagne
134 2017-01-14-cholet
135 2017-01-14-monaco
136 2017-01-14-villeurbanne
137 2017-01-15-pau-orthez
138 2017-01-16-gravelines
139 2017-01-20-hyeres-toulon
140 2017-01-20-le-portel
141 2017-01-21-chalon
142 2017-01-21-le-mans
143 2017-01-21-nancy
144 2017-01-21-nanterre
145 2017-01-21-orleans
146 2017-01-22-limoges
147 2017-01-23-strasbourg
148 2017-02-03-paris-levallois
149 2017-02-03-villeurbanne
150 2017-02-04-gravelines
151 2017-02-04-hyeres-toulon
152 2017-02-04-monaco
153 2017-02-04-nancy
154 2017-02-04-strasbourg
155 2017-02-05-pau-orthez
156 2017-02-06-cholet
157 2017-02-10-antibes
158 2017-02-10-dijon
159 2017-02-10-le-portel
160 2017-02-11-chalon
161 2017-02-11-le-mans
162 2017-02-11-limoges
163 2017-02-11-orleans
164 2017-02-12-gravelines
165 2017-02-13-chalons-en-champagne
166 2017-02-24-hyeres-toulon
167 2017-02-25-cholet
168 2017-02-25-limoges
169 2017-02-25-monaco
170 2017-02-25-nanterre
171 2017-02-25-orlea

132 2017-01-13-cska-moscow
133 2017-01-13-galatasaray
134 2017-01-13-panathinaikos
135 2017-01-13-real-madrid
136 2017-01-19-barcelona
137 2017-01-19-cska-moscow
138 2017-01-19-maccabi-tel-aviv
139 2017-01-19-milano
140 2017-01-19-ulker-fenerbahce
141 2017-01-20-darussafaka
142 2017-01-20-olympiakos
143 2017-01-20-zalgiris
144 2017-01-24-galatasaray
145 2017-01-24-panathinaikos
146 2017-01-24-unics-kazan
147 2017-01-24-vitoria
148 2017-01-24-zalgiris
149 2017-01-25-anadolu-efes
150 2017-01-25-brose-baskets
151 2017-01-25-milano
152 2017-01-26-maccabi-tel-aviv
153 2017-01-26-red-star
154 2017-01-26-ulker-fenerbahce
155 2017-01-27-barcelona
156 2017-01-27-cska-moscow
157 2017-01-27-darussafaka
158 2017-01-27-olympiakos
159 2017-01-27-real-madrid
160 2017-02-02-brose-baskets
161 2017-02-02-milano
162 2017-02-02-ulker-fenerbahce
163 2017-02-02-unics-kazan
164 2017-02-02-zalgiris
165 2017-02-03-galatasaray
166 2017-02-03-red-star
167 2017-02-03-vitoria
168 2017-02-09-anadolu-efes
169 2017-0

In [5]:
player_df = pd.concat(player_dfs_list, ignore_index=True)

In [6]:
len(player_df)

30541

In [7]:
player_df.head()

,player_id,mp,fg,fga,fg3,fg3a,ft,fta,orb,trb,...,tov,pf,pts,player_name,team_id,opp_id,team_pts,opp_pts,boxscore_id,is_playoffs
0,devyn-marble-1,29.0,5.0,16.0,1.0,9.0,6.0,6.0,0.0,5.0,...,0.0,6.0,17.0,Devyn Marble,aris,aries-trikala,75,65,2016-10-08-aris,False
1,spiros-mourtos-1,28.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,...,2.0,1.0,2.0,Spyros Mourtos,aris,aries-trikala,75,65,2016-10-08-aris,False
2,will-cummings-1,21.0,4.0,7.0,1.0,2.0,6.0,6.0,0.0,1.0,...,4.0,5.0,15.0,Will Cummings,aris,aries-trikala,75,65,2016-10-08-aris,False
3,eric-buckner-1,20.0,0.0,2.0,0.0,0.0,3.0,6.0,1.0,8.0,...,1.0,4.0,3.0,Eric Buckner,aris,aries-trikala,75,65,2016-10-08-aris,False
4,michalis-tsairelis-1,20.0,3.0,5.0,0.0,1.0,2.0,2.0,0.0,4.0,...,2.0,1.0,8.0,Michalis Tsairelis,aris,aries-trikala,75,65,2016-10-08-aris,False


### Check data

In [14]:
player_df['team_id'].unique()

array(['aris', 'aries-trikala', 'doxa-bc', 'paok', 'koroivos',
       'olympiakos', 'lavrio', 'kymis-bc', 'rhodes', 'rethymno',
       'apollon-patras', 'aek-athens', 'panathinaikos', 'promitheas',
       'alba-berlin', 'fuenlabrada', 'gran-canaria', 'cedevita', 'khimki',
       'vilnius', 'lietkabelis', 'mzt-skopje', 'murcia', 'buducnost',
       'olimpija-ljubljana', 'kuban', 'triumph-moscow', 'bayern-muenchen',
       'valencia', 'ulm', 'novgorod', 'bilbao', 'unicaja-malaga',
       'hapoel-jerusalem', 'real-madrid', 'estudiantes', 'sevilla',
       'obradoiro', 'vitoria', 'canarias', 'manresa', 'barcelona',
       'joventut', 'zaragoza', 'andorra', 'avellino', 'turin', 'brindisi',
       'trento', 'cantu', 'venezia', 'caserta', 'reggio-emilia',
       'orlandina', 'milano', 'cremona', 'pistoia', 'sassari', 'varese',
       'pesaro', 'brescia', 'hyeres-toulon', 'gravelines', 'le-portel',
       'nanterre', 'paris-levallois', 'nancy', 'chalon', 'orleans',
       'chalons-en-champagne

In [12]:
player_df['opp_id'].unique()

array(['aries-trikala', 'aris', 'paok', 'doxa-bc', 'olympiakos',
       'koroivos', 'kymis-bc', 'lavrio', 'rethymno', 'rhodes',
       'aek-athens', 'apollon-patras', 'promitheas', 'panathinaikos',
       'fuenlabrada', 'alba-berlin', 'cedevita', 'gran-canaria', 'vilnius',
       'khimki', 'mzt-skopje', 'lietkabelis', 'buducnost', 'murcia',
       'kuban', 'olimpija-ljubljana', 'bayern-muenchen', 'triumph-moscow',
       'ulm', 'valencia', 'novgorod', 'bilbao', 'unicaja-malaga',
       'hapoel-jerusalem', 'real-madrid', 'sevilla', 'estudiantes',
       'vitoria', 'obradoiro', 'canarias', 'barcelona', 'manresa',
       'joventut', 'zaragoza', 'andorra', 'turin', 'avellino', 'trento',
       'brindisi', 'venezia', 'cantu', 'reggio-emilia', 'caserta',
       'milano', 'orlandina', 'pistoia', 'cremona', 'varese', 'sassari',
       'pesaro', 'brescia', 'gravelines', 'hyeres-toulon', 'nanterre',
       'le-portel', 'nancy', 'paris-levallois', 'orleans', 'chalon',
       'antibes', 'chalons-e

### Save to csv

In [15]:
player_df.to_csv('raw_data/raw_data.csv', index=False)